# Lab09-4

Model Application (root)

In [1]:
using Cropbox

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16806626827108584244\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-18399678617388667171\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16806626827108584244\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Root Model

Although Cropbox framework was originally intended for implementing traditional process-based models with mostly static structure, we can use `produce` kind of variables for dynamically adding new systems on demand, which is useful for implementing Functional-Structural Plant Models (FSPM). In this example, we implement a root structure growth algorithm used in [CRootBox](https://plant-root-soil-interactions-modelling.github.io/CRootBox/) model.

In [2]:
include("root/root.jl")

Main.Root

### Configurations

Root system architecture in this example constists of three types of segments: primary root, first order lateral root, and second order lateral root. The number of initial primary roots is set by a parameter (`maxB`). As a segment grows, a new segment can be initiated and spawned depending on the transition table (`T`). Each segment type has its own set of parameters related to length (`lb` for basal zone, `la` for axial zone, `ln` for branching interval), elongation rate (`r`), branching angle (`θ`) and so on. Note that some parameter values are combined with standard deviation after `±` operator whose acutal value is decided on initialization. Random sampling of parameters is what gives more realistic root growth behavior for the model.

In [3]:
root_maize = @config(
    :RootArchitecture => :maxB => 5,
    :BaseRoot => :T => [
        # P F S
          0 1 0 ; # P
          0 0 1 ; # F
          0 0 0 ; # S
    ],
    :PrimaryRoot => (
        :lb => 0.1 ± 0.01,
        :la => 18.0 ± 1.8,
        :ln => 0.6 ± 0.06,
        :lmax => 89.7 ± 7.4,
        :r => 6.0 ± 0.6,
        :Δx => 0.5,
        :σ => 10,
        :θ => 80 ± 8,
        :N => 1.5,
        :a => 0.04 ± 0.004,
        :color => Root.RGBA(1, 0, 0, 1),
    ),
    :FirstOrderLateralRoot => (
        :lb => 0.2 ± 0.04,
        :la => 0.4 ± 0.04,
        :ln => 0.4 ± 0.03,
        :lmax => 0.6 ± 1.6,
        :r => 2.0 ± 0.2,
        :Δx => 0.1,
        :σ => 20,
        :θ => 70 ± 15,
        :N => 1,
        :a => 0.03 ± 0.003,
        :color => Root.RGBA(0, 1, 0, 1),
    ),
    :SecondOrderLateralRoot => (
        :lb => 0,
        :la => 0.4 ± 0.02,
        :ln => 0,
        :lmax => 0.4,
        :r => 2.0 ± 0.2,
        :Δx => 0.1,
        :σ => 20,
        :θ => 70 ± 10,
        :N => 2,
        :a => 0.02 ± 0.002,
        :color => Root.RGBA(0, 0, 1, 1),
    )
)

Config for 5 systems:
  RootArchitecture
    maxB = 5
  BaseRoot
    T = [0 1 0; 0 0 1; 0 0 0]
  PrimaryRoot
    lb    = 0.1 ± 0.01
    la    = 18.0 ± 1.8
    ln    = 0.6 ± 0.06
    lmax  = 89.7 ± 7.4
    r     = 6.0 ± 0.6
    Δx    = 0.5
    σ     = 10
    θ     = 80 ± 8
    N     = 1.5
    a     = 0.04 ± 0.004
    color = RGBA{N0f8}(1.0,0.0,0.0,1.0)
  FirstOrderLateralRoot
    lb    = 0.2 ± 0.04
    la    = 0.4 ± 0.04
    ln    = 0.4 ± 0.03
    lmax  = 0.6 ± 1.6
    r     = 2.0 ± 0.2
    Δx    = 0.1
    σ     = 20
    θ     = 70 ± 15
    N     = 1
    a     = 0.03 ± 0.003
    color = RGBA{N0f8}(0.0,1.0,0.0,1.0)
  SecondOrderLateralRoot
    lb    = 0
    la    = 0.4 ± 0.02
    ln    = 0
    lmax  = 0.4
    r     = 2.0 ± 0.2
    Δx    = 0.1
    σ     = 20
    θ     = 70 ± 10
    N     = 2
    a     = 0.02 ± 0.002
    color = RGBA{N0f8}(0.0,0.0,1.0,1.0)

In [4]:
container_pot = @config :Pot => (
    :r1 => 10,
    :r2 => 6,
    :height => 30,
)

Config for 1 systems:
  Pot
    r1     = 10
    r2     = 6
    height = 30

In [5]:
soilcore = @config :SoilCore => (
    :d => 5,
    :l => 20,
    :x0 => 3,
    :y0 => 3,
)

Config for 1 systems:
  SoilCore
    d  = 5
    l  = 20
    x0 = 3
    y0 = 3

### Simulation

First we create an instance of pot (`Root.Pot`) which is a container holding our root system.

In [6]:
b = instance(Root.Pot, config=container_pot)

Pot
  context = <Context>
  config = <Config>
  dist (distance) = <call>
  r1 (top_radius) = 10.0 cm
  r2 (bottom_radius) = 6.0 cm
  h (height) = 30.0 cm
  sq (square) = false

Then, an instance of root system (`Root.RootArchitecture`) is created with parameters for maize (`root_maize`). The instance of pot (`b`) just created above is supplied as an option. `seed` argument indicates a seed number for random number generator. By having a fixed seed number, we have same simulation results every time the model runs.

In [7]:
s = instance(Root.RootArchitecture; config=root_maize, options=(; box=b), seed=0)

RootArchitecture
  context = <Context>
  config = <Config>
  box = <Pot>
  maxB (number_of_basal_roots) = 5
  RT0 (initial_transformation) = IdentityTransformation()
  roots = Main.Root.PrimaryRoot[]

Note that we explicitly create an instance of the model using `instance()` instead of `simulate()` which only gives an output of simulation. We need an instance later used for visualization.

In [8]:
r = simulate!(s, stop=500);

Progress: 100%[=========================================] Time: 0:01:30


We can *gather* all root segments produced during simulation from the current instance of model.

In [9]:
L = gather!(s, Root.BaseRoot; callback=Root.gatherbaseroot!)

161338-element Array{Any,1}:
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 <PrimaryRoot>
 ⋮
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <SecondOrderLateralRoot>
 <FirstOrderLateralRoot>

Then, here is total length of the root.

In [10]:
[s.length' for s in L] |> sum

1430.2434799249459 cm

### Visualization

Due to limitations of JupyterHub, you won't be able to see visualization of 3D root structure in this notebook. Instead, a screenshot captured on a local machine running the same simulation is attached below.

In [11]:
Root.render(s)

![](root/root.png)